In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np


# Set Plotly default template for better aesthetics
# Set template to 'plotly_white' or 'seaborn'
pio.templates.default = "plotly_white"

# --- 1. DATA LOADING AND INITIAL EXPLORATION ---

# Assuming the CSV file "Confectionary [4564].xlsx - Sheet1.csv"
# has been uploaded to the Colab environment.

FILE_PATH = '/content/Confectionary [4564].xlsx'

try:
    # Changed from pd.read_csv to pd.read_excel as the file is an .xlsx
    df = pd.read_excel(FILE_PATH)
    print("Data loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at {FILE_PATH}. Please upload the Excel file to Colab.")
    exit()

# Display initial data info (Task 1: Exploration)
print("\n--- Initial Data Information ---")
print(df.head())
print(df.info())
print(df.describe())

# --- 2. DATA CLEANING AND PREPROCESSING (Task 1: Cleaning) ---

# Rename 'Country(UK)' column for easier access and clarity
df.rename(columns={'Country(UK)': 'Region'}, inplace=True)

# Convert 'Date' column to datetime objects
df['Date'] = pd.to_datetime(df['Date'])

# Clean up column names: remove symbols and extra spaces
df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '', regex=True)
df.rename(columns={'UnitsSold': 'Units_Sold', 'Cost': 'Cost_GBP', 'Profit': 'Profit_GBP', 'Revenue': 'Revenue_GBP'}, inplace=True)

# Calculate the Profit Margin (%) - Essential derived metric
# Profit Margin = (Profit / Revenue) * 100. Handle potential division by zero.
df['Profit_Margin_Pct'] = (df['Profit_GBP'] / df['Revenue_GBP']) * 100
# Cap margin at 100% just in case of erroneous data (though unlikely here)
df['Profit_Margin_Pct'] = df['Profit_Margin_Pct'].clip(upper=100)

print("\n--- Data Cleaning Complete. New Columns & Info ---")
print(df.head())
print(df.info())

# Correcting the confectionary name .
df['Confectionary'] = df['Confectionary'].str.title().str.strip()
df['Confectionary'] = df['Confectionary'].replace({'Choclate Chunk' : 'Chocolate Chunk','Caramel nut':'Caramel Nut'})

# --- 3. ANALYSIS AND VISUALIZATION (5 Distinct Types) ---

# --- Visualisation 1: Regional Profit Comparison (Box Plot) ---
# Objective: Show the distribution, median, and spread of profit margins by region.

print("\n--- Generating Visual 1: Regional Profit Margin (Box Plot) ---")

fig_1_boxplot = px.box(
    df,
    x='Region',
    y='Profit_Margin_Pct',
    color='Region',
    title='1. Regional Profit Margin Distribution (Box Plot)',
    labels={'Profit_Margin_Pct': 'Profit Margin (%)', 'Region': 'UK Region'},
    notched=True,
    height=550
)
fig_1_boxplot.update_traces(quartilemethod="exclusive")
fig_1_boxplot.update_layout(yaxis_title="Profit Margin (%)")
fig_1_boxplot.show()

# --- Visualisation 2: Sales Over Time (Line Chart) ---
# Objective: Identify regional sales trends and peak periods.

print("\n--- Generating Visual 2: Regional Revenue Over Time (Line Chart) ---")

# Aggregate data by month and region for smoother time-series plotting
df_time = df.set_index('Date').groupby([pd.Grouper(freq='ME'), 'Region'])['Revenue_GBP'].sum().reset_index()

fig_2_line = px.line(
    df_time,
    x='Date',
    y='Revenue_GBP',
    color='Region',
    line_shape='spline',
    title='2. Regional Revenue Over Time (Monthly Line Chart)',
    labels={'Revenue_GBP': 'Revenue (£)', 'Date': 'Date'},
    height=600
)
fig_2_line.update_layout(
    yaxis_tickformat='£,d',
    hovermode="x unified"
)
fig_2_line.show()

# --- Visualisation 3: Product Profitability (Stacked Bar Chart) ---
# Objective: Show product contribution to total profit within each region.

print("\n--- Generating Visual 3: Product Profitability (Bar Chart) ---")

# Aggregate total profit by Region and Confectionary type
df_product_profit = df.groupby(['Region', 'Confectionary'])['Profit_GBP'].sum().reset_index()

fig_3_bar = px.bar(
    df_product_profit,
    x='Region',
    y='Profit_GBP',
    color='Confectionary',
    title='3. Total Profit Contribution by Product and Region (Stacked Bar Chart)',
    labels={'Profit_GBP': 'Total Profit (£)', 'Region': 'UK Region'},
    height=550
)
fig_3_bar.update_layout(
    barmode='stack',
    yaxis_tickformat='£,d'
)
fig_3_bar.show()

# --- Visualisation 4: Units Sold vs. Profit (Scatter Plot) ---
# Objective: Examine the relationship between volume (units) and profit.

print("\n--- Generating Visual 4: Units Sold vs. Profit (Scatter Plot) ---")

# Group data by Region and Product to avoid plotting hundreds of individual transactions
df_scatter = df.groupby(['Region', 'Confectionary']).agg(
    Total_Units=('Units_Sold', 'sum'),
    Total_Profit=('Profit_GBP', 'sum'),
    Avg_Revenue_Per_Unit=('Revenue_GBP', lambda x: (x.sum() / df.loc[x.index, 'Units_Sold'].sum())) # Calculate price proxy
).reset_index()

fig_4_scatter = px.scatter(
    df_scatter,
    x='Total_Units',
    y='Total_Profit',
    color='Region', # Color by region for quick comparison
    size='Avg_Revenue_Per_Unit', # Use revenue per unit as size proxy
    hover_name='Confectionary',
    title='4. Units Sold vs. Total Profit (Scatter Plot)',
    labels={'Total_Units': 'Total Units Sold', 'Total_Profit': 'Total Profit (£)'},
    height=600
)
fig_4_scatter.update_layout(
    xaxis_title='Total Units Sold',
    yaxis_title='Total Profit (£)',
    yaxis_tickformat='£,d',
    legend_title='Region'
)
fig_4_scatter.show()

# --- Visualisation 5: Granular Profit Margin (Heatmap) ---
# Objective: Visually find the highest/lowest average margin combinations.

print("\n--- Generating Visual 5: Average Profit Margin (Heatmap) ---")

# Aggregate the mean profit margin by Confectionary and Region
df_heatmap = df.groupby(['Confectionary', 'Region'])['Profit_Margin_Pct'].mean().reset_index()

# Pivot the table to be suitable for a heatmap
heatmap_data = df_heatmap.pivot(index='Confectionary', columns='Region', values='Profit_Margin_Pct')

fig_5_heatmap = px.imshow(
    heatmap_data,
    text_auto=".1f", # Show values formatted to 1 decimal place
    aspect="auto",
    color_continuous_scale=px.colors.sequential.Inferno, # Use a strong color scale to highlight differences
    title='5. Average Profit Margin (%) by Product and Region (Heatmap)',
    labels={'x': 'UK Region', 'y': 'Confectionary Type', 'color': 'Avg. Margin (%)'},
    height=500
)
fig_5_heatmap.update_layout(
    xaxis_nticks=len(heatmap_data.columns) # Ensure all region names are visible
)
fig_5_heatmap.show()

# --- 4. DASHBOARD COMPONENTS (Summary Metrics) ---

total_revenue = df['Revenue_GBP'].sum()
avg_profit_margin = df['Profit_Margin_Pct'].mean()
total_units_sold = df['Units_Sold'].sum()

print("\n--- Summary Metrics for Dashboard/Report ---")
print(f"Total Revenue: £{total_revenue:,.2f}")
print(f"Average Profit Margin: {avg_profit_margin:.2f}%")
print(f"Total Units Sold: {total_units_sold:,.0f}")
print("Take screenshots of the 5 visuals and the dashboard summary for your report.")


Data loaded successfully.

--- Initial Data Information ---
        Date Country(UK) Confectionary  Units Sold  Cost(£)  Profit(£)  \
0 2002-11-11     England       Biscuit      1118.0   2459.6     3130.4   
1 2002-07-05     England       Biscuit       708.0   1557.6     1982.4   
2 2001-10-31     England       Biscuit      1269.0   2791.8     3553.2   
3 2004-09-13     England       Biscuit      1631.0   3588.2     4566.8   
4 2004-03-10     England       Biscuit      2240.0   4928.0     6272.0   

   Revenue(£)  
0    749954.4  
1    300758.4  
2    966216.6  
3   1596096.6  
4   3010560.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           1001 non-null   datetime64[ns]
 1   Country(UK)    1001 non-null   object        
 2   Confectionary  1001 non-null   object        
 3   Units Sold     996 non-null    fl


--- Generating Visual 2: Regional Revenue Over Time (Line Chart) ---



--- Generating Visual 3: Product Profitability (Bar Chart) ---



--- Generating Visual 4: Units Sold vs. Profit (Scatter Plot) ---



--- Generating Visual 5: Average Profit Margin (Heatmap) ---



--- Summary Metrics for Dashboard/Report ---
Total Revenue: £2,521,968,763.65
Average Profit Margin: 0.84%
Total Units Sold: 1,626,827
Take screenshots of the 5 visuals and the dashboard summary for your report.
